In [ ]:
from epx import Job, ModelConfig, SynthPop
import time
import pandas as pd

import os
from pathlib import Path
os.environ["FRED_PROJECT"] = str(Path.cwd())

# Ground logistics model

This model represents a ground transportation network. Cities represent nodes in the network and roads are edges. A table of trucks agents is read in at the beginning of the simulation. There are no human agents in the model. 

## Model components

- `city_locations.fred`: contains the GENERATE\_LOCATIONS condition that creates a network of cities
- `trucks.fred`: contains the GENERATE\_TRUCKS condition that reads the trucks agent file and the MOVE\_TRUCKS condition that picks next locations for trucks to travel to
- `travel_disruption.fred`: contains the DISRUPTION condition that creates travel disruptions at certain nodes as defined by an end user

In [ ]:
ground_shipping_logistics_config = ModelConfig(
             synth_pop=SynthPop("US_2010.v5", ["None"]),
             start_date="2020-01-01",
             end_date="2020-02-01",
         )

ground_shipping_logistics_job = Job(
    "model/main.fred",
    config=[ground_shipping_logistics_config],
    key="cl_ground_shipping_logistics",
    fred_version="11.0.1",
    results_dir="/home/epx/cl-results",
    size="hot"
)


ground_shipping_logistics_job.execute()


# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 300 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(ground_shipping_logistics_job.status) != 'DONE':
    if str(ground_shipping_logistics_job.status) == 'ERROR':
        logs = ground_shipping_logistics_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(ground_shipping_logistics_job.status)

## Data outputs

Let's take a look at the output `travel_info.csv`. The data summarizes the progress of each truck on the journey from its most recently visited city to its next destination by recording the remaining distance left to travel on each day of the simulation.

In [ ]:
travel_log_df = ground_shipping_logistics_job.results.csv_output('travel_log.csv')
travel_log_df

In [ ]:
ground_shipping_logistics_job.delete(interactive=False)